## Description: Medical RAG Data Ingestion and Vector DB Creation
## Author: "Mostafa Shams"

In [1]:
!pip install -q langchain-community langchain-huggingface chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 8.8 MB/s eta 0:00

In [2]:
import json
import os

FILE_PATH = "/kaggle/input/medicalrag/medical_rag.json"

if not os.path.exists(FILE_PATH):
    print(f"File not found at {FILE_PATH}")
else:
    print(f"File found at {FILE_PATH}")

    with open(FILE_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)

    print(f"Total Documents Loaded: {len(data)}")

    first_doc_content = data[10]['content']
    word_count = len(first_doc_content.split())
    char_count = len(first_doc_content)
    
    print(f"Document Inspection:")
    print(f"File Name: {data[10].get('file_name', 'Unknown')}")
    print(f"Word Count: {word_count} words")
    print(f"Character Count: {char_count} characters")
    
    if word_count > 400:
        print("Document is too long, requires splitting.")
    else:
        print("Document fits within limits.")

✅ File found at /kaggle/input/medicalrag/medical_rag.json

📊 Total Documents Loaded: 279

--- 🧐 INSPECTION OF DOCUMENT #1 ---
File Name: Feeding%20Baby%20Before%2038%20wks%20ARBupdated.pdf
Word Count: 775 words
Character Count: 4193 characters
⚠️ ALERT: This document is too long for a single embedding. It requires splitting.


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNK_SIZE = 1000
CHUNK_OVERLAP = 130

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ".", "،", " ", ""] 
)

print("Text Splitter Configured.")
print(f"Target Chunk Size: {CHUNK_SIZE} characters")
print(f"Overlap: {CHUNK_OVERLAP} characters")

2025-11-21 19:02:28.798141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763751749.011891      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763751749.068791      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

✂️ Text Splitter Configured.
Target Chunk Size: 1000 characters
Overlap: 130 characters


In [4]:
from langchain.docstore.document import Document

documents = []

for item in data:
    doc = Document(
        page_content=item["content"],
        metadata=item["metadata"] if "metadata" in item else {}
    )
    documents.append(doc)

split_docs = text_splitter.split_documents(documents)

print(f"Splitting Report:")
print(f"Original Documents: {len(documents)}")
print(f"Resulting Chunks:   {len(split_docs)}")
print(f"Expansion Factor:   {len(split_docs)/len(documents):.1f}x")

for i, chunk in enumerate(split_docs[2:8]): 
    print(f"Chunk {i+1}:")
    print(f"Length: {len(chunk.page_content)} characters")
    print(f"Metadata: {chunk.metadata}")
    print(chunk.page_content) 
    print("-" * 20)

print("Checking Overlap:")
print(f"End of Chunk 1: '...{split_docs[0].page_content[-200:]}'")
print(f"Start of Chunk 2: '{split_docs[1].page_content[:200]}...'")


🔄 SPLITTING REPORT
Original Documents: 279
Resulting Chunks:   1330
Expansion Factor:   4.8x (Average chunks per document)

👀 VISUALIZING THE SPLIT (First 3 Chunks)

🔹 CHUNK #1
Length: 567 characters
Metadata: {'source': 'MedlinePlus', 'type': 'health'}
--------------------
بعد الولادة، ستقوم إحدى الممرضات بتفقد حالتك الصحية واإلجابة على كافة تساؤالتك وأثناء فترة إقامتك، سوف تعلمك الممرضات كيفية الاعتناء بنفسك وبطفلك التغييرات التي تطرأ على جسدك ستقوم إحدى الممرضات بتحسس منطقة البطن لديك لتفقد حالة الرحم أثناء فترة تواجدك في المستشفى إذا كان الرحم مرنًا، ستقوم الممرضة بحك بطنك للمساعدة على زيادة صلابة الرحم من الطبيعي أن يكون لديكِ تقلصات في الرحم تسمى آلام ما بعد الولادة غالبًا ما يستمر هذا الأمر لعدة أيام وقد تزداد سوءًا أثناء الرضاعة الطبيعية ستنزفين من المهبل لمدة تتراوح من أسبوعين إلى أربعة أسابيع وأثناء الأيام القليلة الأولى
--------------------

🔹 CHUNK #2
Length: 835 characters
Metadata: {'source': 'MedlinePlus', 'type': 'health'}
--------------------
، يشبه النزيف دورة حيض شه

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

DB_DIR = "./medical_rag_db"

print("Loading Embedding Model (intfloat/multilingual-e5-large)...")
embedding_model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={'device': 'cuda'} 
)

print("Creating Vector Database...")
vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    persist_directory=DB_DIR
)

print(f"Database successfully created at: {DB_DIR}")
print(f"Total Chunks Stored: {vectordb._collection.count()}")

🚀 Loading Embedding Model (intfloat/multilingual-e5-large)...


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

💾 Creating Vector Database...
✅ Database successfully created at: ./medical_rag_db
Total Chunks Stored: 1330


In [6]:
query = "كم مرة يجب إرضاع الطفل؟"
print(f"Testing Query: {query}")
print("-" * 50)

results_with_scores = vectordb.similarity_search_with_score(query, k=20)

print(f"Found {len(results_with_scores)} relevant chunks:\n")

for i, (doc, score) in enumerate(results_with_scores):
    
    print(f"Result #{i+1}")
    print(f"Confidence Score (Distance): {score:.4f}") 
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content:")
    print(f"{doc.page_content}") 
    print("\n" + "="*50 + "\n")

❓ Testing Query: كم مرة يجب إرضاع الطفل؟
--------------------------------------------------

Found 20 relevant chunks:

🔹 Result #1
🎯 Confidence Score (Distance): 0.2715
📄 Source: MedlinePlus
📝 Content:
، فسوف يكون ثدييك ممتلئين بالحليب، يصبحان لينين بعد الرضاعة، ثم يمتلئان مرة أخرى بين مرات التغذية سوف: يقوم طفلك بما يلي ساعة 24 حفاضات أو أكثر خالل 6 استهالك النوم بين الرضعات سيكون لديه أكثر من حركتين لألمعاء كل يوم يزداد وزنه ال يحتاج الطفل الذي يتمتع بصحة جيدة ويحصل على رضاعة طبيعية إلى الحصول على مياه أو حليب اصطناعي إضافي ال تعطي طفلك مياه أو حليب اصطناعي إضافي ما لم يأمر طبيب الطفل بذلك تحدثي إلى طبيب طفلك أو الممرضة أو أخصائية الرضاعة إذا كنتِ ال تعتقدين أن طفلك يحصل على ما يكفيه من الحليب كم مرة سيتغذى طفلي؟ يتم تحديد توقيت التغذية من بداية رضعة واحدة إلى بداية الرضاعة التالية في البداية، حاولي إرضاع طفلك كل 4 ساعتين إلى ثالث ساعات خلال النهار والمساء، وكل ساعات على الأقل أثناء الليل وإذا أظهر طفلك إشارات الحاجة إلى التغذية، فقومي بإرضاعه قد يحتاج طفلك عدة رضعات قريبة جدًا من ب